In [1]:
import pandas as pd

from baseball_scraper import playerid_lookup
from baseball_scraper import statcast_pitcher
from baseball_scraper import batting_stats_range
from baseball_scraper import statcast
from baseball_scraper import statcast_batter

from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('./dummy_statcast_2015-2020.csv')

/Users/johnnybex/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 250)

In [4]:
df.head()

,Unnamed: 0,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,events_batter_interference,events_caught_stealing_2b,events_caught_stealing_3b,events_caught_stealing_home,events_double,events_double_play,events_field_error,events_field_out,events_fielders_choice,events_fielders_choice_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_intent_walk,events_interf_def,events_other_out,events_pickoff_1b,events_pickoff_2b,events_pickoff_3b,events_pickoff_caught_stealing_2b,events_pickoff_caught_stealing_3b,events_pickoff_caught_stealing_home,events_run,events_sac_bunt,events_sac_bunt_double_play,events_sac_fly,events_sac_fly_double_play,events_single,events_strikeout,events_strikeout_double_play,events_triple,events_triple_play,events_walk,hard,soft
0,0,0,FF,2020-10-27,96.7,1.58,5.99,Julio Urias,642715,628711.0,called_strike,NaN,NaN,NaN,NaN,4.0,Willy Adames called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,0.0,2.0,2020.0,0.18,1.63,-0.53,2.29,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-5.950264,-140.490456,-7.897391,3.772000,32.321911,-8.981441,3.50,1.69,NaN,NaN,NaN,95.4,2615.0,5.7,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.82,NaN,NaN,0.0,1.0,0.0,0.0,NaN,65.0,3.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,9,9,FF,2020-10-27,95.2,1.72,6.09,Julio Urias,622534,628711.0,hit_into_play,NaN,NaN,NaN,NaN,6.0,Manuel Margot flies out to right fielder Mooki...,W,R,L,LAD,TB,X,9.0,fly_ball,1.0,2.0,2020.0,0.40,1.48,0.40,2.56,NaN,NaN,NaN,0.0,9.0,Top,191.34,99.03,NaN,NaN,605131.0,NaN,NaN,-4.306764,-138.539023,-6.867868,6.166594,30.824839,-11.731952,3.25,1.53,284.0,83.4,44.0,94.0,2450.0,5.6,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.92,0.029,0.040,0.0,1.0,0.0,0.0,3.0,63.0,4.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,13,13,SI,2020-10-27,87.3,2.80,5.62,Ryan Yarbrough,571970,642232.0,hit_into_play,NaN,NaN,NaN,NaN,5.0,"Max Muncy grounds into a double play, second b...",W,L,L,LAD,TB,X,4.0,ground_ball,1.0,0.0,2020.0,1.09,0.72,0.05,2.50,NaN,NaN,608369.0,1.0,8.0,Bot,155.53,145.59,NaN,NaN,605421.0,NaN,NaN,-8.876187,-126.909224,-2.863199,13.515807,24.416540,-23.996752,3.11,1.52,66.0,97.4,2.0,87.7,1914.0,6.4,635886.0,642232.0,605421.0,650490.0,664040.0,621563.0,642715.0,668227.0,595281.0,622534.0,54.06,0.464,0.418,0.0,1.0,0.0,0.0,4.0,62.0,2.0,Sinker,3.0,1.0,3.0,1.0,1.0,3.0,3.0,1.0,Infield shift,Standard,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,15,15,FF,2020-10-27,98.0,0.07,6.77,Pete Fairbanks,571771,664126.0,swinging_strike,NaN,NaN,NaN,NaN,12.0,Enrique Hernandez strikes out swinging.,W,R,R,LAD,TB,S,2.0,NaN,1.0,2.0,2020.0,-0.05,1.42,0.52,3.51,NaN,NaN,608369.0,0.0,8.0,Bot,NaN,NaN,NaN,NaN,605421.0,NaN,NaN,1.363512,-142.729662,-6.710995,-1.015568,31.662677,-11.390841,3

In [5]:
#df = df.drop(columns='Unnamed: 0')

In [6]:
df = df.drop(columns=['soft', 'hard'])

In [7]:
# not including intentional ball, pitchout, unknown
fast = ['4-Seam Fastball', '2-Seam Fastball', 'Sinker', 'Cutter', 'Fastball']
change = ['Changeup']
slider = ['slider']
curve = ['Curveball', 'Knuckle Curve']
junk = ['Split-Finger', 'Knuckleball', 'Eephus', 'Forkball', 'Screwball']

df['fast'] = [1 if x in fast else 0 for x in df['pitch_name']]
df['change'] = [1 if x in change else 0 for x in df['pitch_name']]
df['slider'] = [1 if x in slider else 0 for x in df['pitch_name']]
df['curve'] = [1 if x in curve else 0 for x in df['pitch_name']]
df['junk'] = [1 if x in junk else 0 for x in df['pitch_name']]

In [8]:
df['risp'] = df['on_2b'] + df['on_3b']

In [9]:
df['risp'] = [1 if x > 0 else 0 for x in (df['risp'])]

In [10]:
df = df.drop(columns=['on_1b', 'on_2b', 'on_3b'])

In [11]:
df.head()

,Unnamed: 0,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,events_batter_interference,events_caught_stealing_2b,events_caught_stealing_3b,events_caught_stealing_home,events_double,events_double_play,events_field_error,events_field_out,events_fielders_choice,events_fielders_choice_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_intent_walk,events_interf_def,events_other_out,events_pickoff_1b,events_pickoff_2b,events_pickoff_3b,events_pickoff_caught_stealing_2b,events_pickoff_caught_stealing_3b,events_pickoff_caught_stealing_home,events_run,events_sac_bunt,events_sac_bunt_double_play,events_sac_fly,events_sac_fly_double_play,events_single,events_strikeout,events_strikeout_double_play,events_triple,events_triple_play,events_walk,fast,change,slider,curve,junk,risp
0,0,0,FF,2020-10-27,96.7,1.58,5.99,Julio Urias,642715,628711.0,called_strike,NaN,NaN,NaN,NaN,4.0,Willy Adames called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,0.0,2.0,2020.0,0.18,1.63,-0.53,2.29,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-5.950264,-140.490456,-7.897391,3.772000,32.321911,-8.981441,3.50,1.69,NaN,NaN,NaN,95.4,2615.0,5.7,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.82,NaN,NaN,0.0,1.0,0.0,0.0,NaN,65.0,3.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,9,9,FF,2020-10-27,95.2,1.72,6.09,Julio Urias,622534,628711.0,hit_into_play,NaN,NaN,NaN,NaN,6.0,Manuel Margot flies out to right fielder Mooki...,W,R,L,LAD,TB,X,9.0,fly_ball,1.0,2.0,2020.0,0.40,1.48,0.40,2.56,0.0,9.0,Top,191.34,99.03,NaN,NaN,605131.0,NaN,NaN,-4.306764,-138.539023,-6.867868,6.166594,30.824839,-11.731952,3.25,1.53,284.0,83.4,44.0,94.0,2450.0,5.6,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.92,0.029,0.040,0.0,1.0,0.0,0.0,3.0,63.0,4.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,13,13,SI,2020-10-27,87.3,2.80,5.62,Ryan Yarbrough,571970,642232.0,hit_into_play,NaN,NaN,NaN,NaN,5.0,"Max Muncy grounds into a double play, second b...",W,L,L,LAD,TB,X,4.0,ground_ball,1.0,0.0,2020.0,1.09,0.72,0.05,2.50,1.0,8.0,Bot,155.53,145.59,NaN,NaN,605421.0,NaN,NaN,-8.876187,-126.909224,-2.863199,13.515807,24.416540,-23.996752,3.11,1.52,66.0,97.4,2.0,87.7,1914.0,6.4,635886.0,642232.0,605421.0,650490.0,664040.0,621563.0,642715.0,668227.0,595281.0,622534.0,54.06,0.464,0.418,0.0,1.0,0.0,0.0,4.0,62.0,2.0,Sinker,3.0,1.0,3.0,1.0,1.0,3.0,3.0,1.0,Infield shift,Standard,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,15,15,FF,2020-10-27,98.0,0.07,6.77,Pete Fairbanks,571771,664126.0,swinging_strike,NaN,NaN,NaN,NaN,12.0,Enrique Hernandez strikes out swinging.,W,R,R,LAD,TB,S,2.0,NaN,1.0,2.0,2020.0,-0.05,1.42,0.52,3.51,0.0,8.0,Bot,NaN,NaN,NaN,NaN,605421.0,NaN,NaN,1.363512,-142.729662,-6.710995,-1.015568,31.662677,-11.390841,3.25,1.58,NaN,NaN,NaN,99.2,2

In [12]:
df['zone'].value_counts(normalize=True)

14.0    0.122624
5.0     0.110410
13.0    0.101618
8.0     0.091995
4.0     0.087928
6.0     0.083228
11.0    0.072809
9.0     0.072368
7.0     0.068382
2.0     0.054142
12.0    0.052083
1.0     0.043546
3.0     0.038867
Name: zone, dtype: float64

In [13]:
df['launch_speed_angle'].value_counts(normalize=True)

2.0    0.339725
4.0    0.252632
3.0    0.250085
6.0    0.067197
5.0    0.058376
1.0    0.031986
Name: launch_speed_angle, dtype: float64

In [14]:
df.drop(columns=['spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated',
                'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated',
                'umpire'], inplace=True)

In [15]:
# 'release_spin_rate', 'effective_speed', 'pitch_number', 
sc_columns = ['batter', 'zone', 'game_type', 'stand', 'p_throws', 
                'hit_location', 'bb_type', 'balls', 'strikes', 'outs_when_up', 
                'inning', 'inning_topbot', 'hc_x', 'hc_y','hit_distance_sc', 
                'launch_speed', 'launch_angle', 'woba_value','babip_value', 
              'iso_value', 'launch_speed_angle', 'risp',  'fast',
             'change', 'slider', 'curve', 'junk']

# Removing events accounted for in wOBA
events = [ 'events_field_out',
'events_fielders_choice', 'events_fielders_choice_out', 'events_force_out', 
'events_grounded_into_double_play','events_other_out','events_run', 'events_sac_bunt', 
'events_sac_fly', 'events_strikeout', 'events_triple_play', 'events_intent_walk']

columns = sc_columns + events

In [16]:
df['game_type']

0         W
1         W
2         W
3         W
4         W
         ..
938809    R
938810    R
938811    R
938812    R
938813    R
Name: game_type, Length: 938814, dtype: object

In [17]:
df = df[columns]

In [18]:
# Play made by catcher left out 
right_side = [3,4,7]
middle = [1,8]
left_side = [5,6,9]

df['right_side_field'] = [1 if x in right_side else 0 for x in df['hit_location']]
df['middle_field'] = [1 if x in middle else 0 for x in df['hit_location']]
df['left_side_field'] = [1 if x in left_side else 0 for x in df['hit_location']]

df = df.drop(columns=['hit_location'])

In [19]:
df = pd.get_dummies(data=df, columns=['bb_type'], drop_first=True)

In [20]:
# Left out spring training and exhibition
df['regular_season'] = [1 if x == 'R' else 0 for x in df['game_type']]

postseason = ['F', 'D', 'L', 'W']

df['postseason'] = [1 if x in postseason else 0 for x in df['game_type']]



In [21]:
df['two_strike'] = [1 if x == 2 else 0 for x in df['strikes']]
df = df.drop(columns=['balls', 'strikes'])

In [22]:
df = pd.get_dummies(data=df, columns=['outs_when_up'], drop_first=True)

In [23]:
# first_third_game = [1 if x <=3 else 0 for x in df['inning']]
# middle_of_game = [1 if x >3 and x < 7 else 0 for x in df['inning']]
# late_game = [1 if x > 7 else 0 for x in df['inning']]

In [24]:
df = pd.get_dummies(data=df, columns=['inning_topbot'], drop_first=True)

In [25]:
# Excluding chase_left
middle_middle = [2, 5, 8]
middle_left = [1, 4, 7]
middle_right = [3, 6, 9]
chase_right = [12, 14]

df['middle_middle'] = [1 if x in middle_middle else 0 for x in df['zone']]
df['middle_left'] = [1 if x in middle_left else 0 for x in df['zone']]
df['middle_right'] = [1 if x in middle_right else 0 for x in df['zone']]
df['chase_right'] = [1 if x in chase_right else 0 for x in df['zone']]

df = df.drop(columns=['zone'])

In [26]:
df = pd.get_dummies(data=df, columns=['launch_speed_angle'], drop_first=True)

In [27]:
df = pd.get_dummies(data=df, columns=['stand', 'p_throws'], drop_first=True)

In [28]:
df.head()

,batter,game_type,inning,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,risp,fast,change,slider,curve,junk,events_double,events_field_out,events_fielders_choice,events_fielders_choice_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_other_out,events_run,events_sac_bunt,events_sac_fly,events_single,events_strikeout,events_triple,events_triple_play,events_walk,right_side_field,middle_field,left_side_field,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,regular_season,postseason,two_strike,outs_when_up_1.0,outs_when_up_2.0,inning_topbot_Top,middle_middle,middle_left,middle_right,chase_right,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,stand_R,p_throws_R
0,642715,W,9.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,1,0
1,622534,W,9.0,191.34,99.03,284.0,83.4,44.0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0
2,571970,W,8.0,155.53,145.59,66.0,97.4,2.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0
3,571771,W,8.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1
4,608369,W,8.0,NaN,NaN,NaN,NaN,NaN,0.7,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [29]:
# value count number of times player in different post-season series.

In [30]:
df['launch_speed'].mean()

88.34529462143225

In [31]:
df['game_type'].value_counts()

R    922249
D      7051
L      4830
W      2730
F      1954
Name: game_type, dtype: int64

In [38]:
#innings = df['inning'].unique()
#df['middle_innings'] = [1 if x in [4,5,6] else 0 for x in df['inning']]
df['late_innings'] = [1 if x in [7,8,9] else 0 for x in df['inning']]

#df.groupby(['batter', 'middle_innings']).mean()
df.groupby(['batter', 'late_innings']).mean()
df = df.drop(columns=['inning'])

In [39]:
df.head()

,batter,game_type,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,risp,fast,change,slider,curve,junk,events_double,events_field_out,events_fielders_choice,events_fielders_choice_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_other_out,events_run,events_sac_bunt,events_sac_fly,events_single,events_strikeout,events_triple,events_triple_play,events_walk,right_side_field,middle_field,left_side_field,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,regular_season,postseason,two_strike,outs_when_up_1.0,outs_when_up_2.0,inning_topbot_Top,middle_middle,middle_left,middle_right,chase_right,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,stand_R,p_throws_R,middle_innings,late_innings
0,642715,W,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,1
1,622534,W,191.34,99.03,284.0,83.4,44.0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1
2,571970,W,155.53,145.59,66.0,97.4,2.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
3,571771,W,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1
4,608369,W,NaN,NaN,NaN,NaN,NaN,0.7,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1


In [48]:
df.game_type.value_counts()

R    922249
D      7051
L      4830
W      2730
F      1954
Name: game_type, dtype: int64

In [57]:
df['post_season_performance'] = [1 if x != 'R' else 0 for x in df['game_type']]

df = df.drop(columns = ['game_type'])

batter
116338     0
120074    12
121347     4
133380     1
134181    22
          ..
668804     0
669222     7
669242    50
669374    11
670032     0
Name: post_season_performance, Length: 694, dtype: int64

In [ ]:
#df.groupby('batter')[['game_type_R','game_type_F', 'game_type_D', 'game_type_L', 'game_type_W']].sum()

In [53]:
df.head()

,batter,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,risp,fast,change,slider,curve,junk,events_double,events_field_out,events_fielders_choice,events_fielders_choice_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_other_out,events_run,events_sac_bunt,events_sac_fly,events_single,events_strikeout,events_triple,events_triple_play,events_walk,right_side_field,middle_field,left_side_field,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,regular_season,postseason,two_strike,outs_when_up_1.0,outs_when_up_2.0,inning_topbot_Top,middle_middle,middle_left,middle_right,chase_right,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,stand_R,p_throws_R,middle_innings,late_innings,post_season_performance
0,642715,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,1,1
1,622534,191.34,99.03,284.0,83.4,44.0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,1
2,571970,155.53,145.59,66.0,97.4,2.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1
3,571771,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1
4,608369,NaN,NaN,NaN,NaN,NaN,0.7,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1


In [ ]:
df['post_season_appearances'] = df.groupby('batter')['post_season_performance'].sum()
df = df.drop('post_season_performance')

In [54]:
df_gb = df

0.0    860900
1.0     42964
3.0     30661
2.0      4289
Name: iso_value, dtype: int64

In [ ]:
df_ss = StandardScaler().fit_transform(df_gb)

In [ ]:
model = KMeans(n_clusters=4, random_state=0)
model.fit(rh_gb_sc)